# Xbox Controller Teleoperation for Lite6 Robot Arm

Real-time control of the 6DOF robot arm using an Xbox controller connected via USB.

## Controller Mapping

**Cartesian Mode** (control end-effector position):
- Left Stick: Move in X-Y plane
- Right Stick Y: Move up/down (Z)
- Right Stick X: Rotate around Z (yaw)
- LB/RB: Pitch rotation
- LT/RT: Roll rotation

**Joint Mode** (control individual joints):
- Left Stick: Joints 1-2
- Right Stick: Joints 3-4
- LB/RB: Joint 5
- LT/RT: Joint 6

**Common**:
- A Button: Switch between modes
- B Button: Emergency stop
- Start Button: Return to home position

## Requirements
```bash
pip install pygame mujoco numpy matplotlib
```

## Setup

In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

import numpy as np
import pygame
import time
import matplotlib.pyplot as plt
from applied_planning.sim.adapters.mujoco_backend import MujocoLite6Adapter

print('✓ Imports successful')

## Xbox Controller Class

In [ ]:
class XboxController:
    '''Xbox controller interface using pygame.'''
    
    def __init__(self, dead_zone=0.15):
        pygame.init()
        pygame.joystick.init()
        
        if pygame.joystick.get_count() == 0:
            raise RuntimeError('No controller detected!')
        
        self.controller = pygame.joystick.Joystick(0)
        self.controller.init()
        self.dead_zone = dead_zone
        
        print(f'✓ Controller: {self.controller.get_name()}')
    
    def get_state(self):
        pygame.event.pump()
        
        left_x = self._deadzone(self.controller.get_axis(0))
        left_y = self._deadzone(self.controller.get_axis(1))
        right_x = self._deadzone(self.controller.get_axis(2))
        right_y = self._deadzone(self.controller.get_axis(3))
        
        try:
            lt = (self.controller.get_axis(4) + 1) / 2
            rt = (self.controller.get_axis(5) + 1) / 2
        except:
            lt, rt = 0, 0
        
        buttons = {
            'A': self.controller.get_button(0),
            'B': self.controller.get_button(1),
            'X': self.controller.get_button(2),
            'Y': self.controller.get_button(3),
            'LB': self.controller.get_button(4),
            'RB': self.controller.get_button(5),
            'Start': self.controller.get_button(7),
        }
        
        return {
            'left_stick': (left_x, left_y),
            'right_stick': (right_x, right_y),
            'triggers': (lt, rt),
            'buttons': buttons
        }
    
    def _deadzone(self, value):
        if abs(value) < self.dead_zone:
            return 0.0
        sign = 1 if value > 0 else -1
        return sign * (abs(value) - self.dead_zone) / (1 - self.dead_zone)
    
    def close(self):
        pygame.quit()

print('✓ XboxController class defined')

## Teleoperation Controller

In [ ]:
class RobotTeleop:
    '''Maps Xbox input to robot motion.'''
    
    def __init__(self, sim, rate=20.0):
        self.sim = sim
        self.dt = 1.0 / rate
        self.cartesian_mode = True
        self.max_cart_vel = 0.15
        self.max_ang_vel = 0.8
        self.max_joint_vel = 1.0
        self.home_qpos = np.array([0, 0, 1.57, 0, 1.57, 0])
        self.e_stop = False
    
    def cartesian_control(self, state):
        lx, ly = state['left_stick']
        rx, ry = state['right_stick']
        lt, rt = state['triggers']
        lb = state['buttons']['LB']
        rb = state['buttons']['RB']
        
        return np.array([
            -ly * self.max_cart_vel,
            lx * self.max_cart_vel,
            -ry * self.max_cart_vel,
            (rt - lt) * self.max_ang_vel,
            (rb - lb) * self.max_ang_vel,
            rx * self.max_ang_vel
        ])
    
    def joint_control(self, state):
        lx, ly = state['left_stick']
        rx, ry = state['right_stick']
        lt, rt = state['triggers']
        lb = state['buttons']['LB']
        rb = state['buttons']['RB']
        
        return np.array([
            ly * self.max_joint_vel,
            -lx * self.max_joint_vel,
            -ry * self.max_joint_vel,
            rx * self.max_joint_vel,
            (rb - lb) * self.max_joint_vel,
            (rt - lt) * self.max_joint_vel
        ])
    
    def cart_to_joint_vel(self, cart_vel):
        import mujoco
        jacp = np.zeros((3, self.sim.model.nv))
        jacr = np.zeros((3, self.sim.model.nv))
        
        try:
            site_id = mujoco.mj_name2id(
                self.sim.model, mujoco.mjtObj.mjOBJ_SITE, self.sim.ee_site_name)
            mujoco.mj_jacSite(self.sim.model, self.sim.data, jacp, jacr, site_id)
        except:
            body_id = self.sim.model.nbody - 1
            mujoco.mj_jacBody(self.sim.model, self.sim.data, jacp, jacr, body_id)
        
        jac = np.vstack([jacp[:, :6], jacr[:, :6]])
        return np.linalg.pinv(jac) @ cart_vel
    
    def go_home(self):
        print('Moving to home...')
        current = self.sim.data.qpos[:6].copy()
        for i in range(100):
            alpha = i / 100
            target = (1-alpha)*current + alpha*self.home_qpos
            self.sim.set_state({'qpos': target, 'qvel': np.zeros(6)})
            time.sleep(0.01)
        self.e_stop = False
        print('✓ Home')

print('✓ RobotTeleop class defined')

## Initialize

In [ ]:
# Initialize robot
model_path = '../src/applied_planning/sim/assets/ufactory_lite6/lite6.xml'
sim = MujocoLite6Adapter(model_path, viewer=False, render_mode='offscreen')
sim.reset()
sim.set_state({'qpos': np.array([0, 0, 1.57, 0, 1.57, 0]), 'qvel': np.zeros(6)})
print('✓ Robot ready')

# Connect controller
xbox = XboxController()
print('✓ Controller ready')

# Initialize teleop
teleop = RobotTeleop(sim, rate=20.0)
print('✓ Teleop ready')

## Test Controller (optional)

In [ ]:
# Test - move sticks to see values
for i in range(30):
    state = xbox.get_state()
    lx, ly = state['left_stick']
    if abs(lx) > 0.01 or abs(ly) > 0.01:
        print(f'Left: ({lx:.2f}, {ly:.2f})', end='\r')
    time.sleep(0.1)
print('\n✓ Test done')

## Main Control Loop

**Press Interrupt Kernel (■) to stop**

Controls:
- A: Switch mode
- B: Emergency stop  
- Start: Go home

In [ ]:
print('='*60)
print('TELEOPERATION ACTIVE')
print('='*60)

last_a = False
loop_count = 0

try:
    while True:
        start_time = time.time()
        state = xbox.get_state()
        
        # Emergency stop
        if state['buttons']['B']:
            if not teleop.e_stop:
                teleop.e_stop = True
                print('\n⚠ EMERGENCY STOP')
        
        # Go home
        if state['buttons']['Start']:
            teleop.go_home()
        
        # Mode switch
        if state['buttons']['A'] and not last_a:
            teleop.cartesian_mode = not teleop.cartesian_mode
            mode = 'Cartesian' if teleop.cartesian_mode else 'Joint'
            print(f'\n→ {mode} mode')
        last_a = state['buttons']['A']
        
        if teleop.e_stop:
            time.sleep(teleop.dt)
            continue
        
        # Compute velocity
        if teleop.cartesian_mode:
            cart_vel = teleop.cartesian_control(state)
            joint_vel = teleop.cart_to_joint_vel(cart_vel)
        else:
            joint_vel = teleop.joint_control(state)
        
        # Apply motion
        sim.step({'qvel': joint_vel})
        loop_count += 1
        
        # Status display
        if loop_count % 20 == 0:
            ee = sim.get_ee_position()
            mode = 'C' if teleop.cartesian_mode else 'J'
            print(f'[{mode}] EE: [{ee[0]:.3f}, {ee[1]:.3f}, {ee[2]:.3f}]', end='\r')
        
        # Maintain rate
        elapsed = time.time() - start_time
        time.sleep(max(0, teleop.dt - elapsed))

except KeyboardInterrupt:
    print('\n' + '='*60)
    print(f'STOPPED after {loop_count} loops ({loop_count*teleop.dt:.1f}s)')
    print('='*60)

## Visualize Current State

In [ ]:
img = sim.render_notebook(width=800, height=600)
plt.figure(figsize=(12, 9))
plt.imshow(img)
plt.axis('off')
plt.title(f'EE: {sim.get_ee_position()}')
plt.show()

print(f'Joints: {sim.data.qpos[:6]}')
print(f'EE pos: {sim.get_ee_position()}')

## Record Trajectory

In [ ]:
print('Recording... Press B to stop')

traj = {'time': [], 'qpos': [], 'ee_pos': []}
start = time.time()
teleop.e_stop = False

try:
    while True:
        state = xbox.get_state()
        if state['buttons']['B']:
            break
        
        if teleop.cartesian_mode:
            vel = teleop.cartesian_control(state)
            jvel = teleop.cart_to_joint_vel(vel)
        else:
            jvel = teleop.joint_control(state)
        
        sim.step({'qvel': jvel})
        
        traj['time'].append(time.time() - start)
        traj['qpos'].append(sim.data.qpos[:6].copy())
        traj['ee_pos'].append(sim.get_ee_position().copy())
        
        if len(traj['time']) % 20 == 0:
            print(f'Samples: {len(traj["time"])}', end='\r')
        
        time.sleep(teleop.dt)
except KeyboardInterrupt:
    pass

print(f'\n✓ Recorded {len(traj["time"])} samples')

## Plot Trajectory

In [ ]:
t = np.array(traj['time'])
qpos = np.array(traj['qpos'])
ee_pos = np.array(traj['ee_pos'])

fig, axes = plt.subplots(2, 1, figsize=(14, 8))

axes[0].set_title('Joint Positions')
for i in range(6):
    axes[0].plot(t, qpos[:, i], label=f'J{i+1}')
axes[0].legend(ncol=6)
axes[0].grid(True, alpha=0.3)

axes[1].set_title('End-Effector Position')
axes[1].plot(t, ee_pos[:, 0], label='X')
axes[1].plot(t, ee_pos[:, 1], label='Y')
axes[1].plot(t, ee_pos[:, 2], label='Z')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 3D plot
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
ax.plot(ee_pos[:, 0], ee_pos[:, 1], ee_pos[:, 2])
ax.scatter(ee_pos[0, 0], ee_pos[0, 1], ee_pos[0, 2], c='green', s=100)
ax.scatter(ee_pos[-1, 0], ee_pos[-1, 1], ee_pos[-1, 2], c='red', s=100)
ax.set_xlabel('X (m)')
ax.set_ylabel('Y (m)')
ax.set_zlabel('Z (m)')
plt.show()

dist = np.sum(np.linalg.norm(np.diff(ee_pos, axis=0), axis=1))
print(f'Duration: {t[-1]:.2f}s')
print(f'Distance: {dist:.3f}m')

## Cleanup

In [ ]:
xbox.close()
sim.close()
print('✓ Cleanup done')